In [ ]:
import os
import platform
import random
import shutil
import sys
import datetime
import copy

# recommended Python3 version >= 3.5
print('Python version: {}'.format(platform.sys.version))

# data-science & processing tools
import numpy as np
import pandas as pd
import sklearn.metrics
import h5py

# progress bar
try:
    from tqdm.notebook import tqdm
except ImportError:
    from tqdm import tqdm_notebook as tqdm

# plotting utilities
import seaborn as sns
import matplotlib.pyplot as plt

# tensorflow
import tensorflow as tf
import tensorflow.keras as K

# required TensorFlow version >= 2.0.0
tf_version = tf.__version__
print('TensorFlow version: {}'.format(tf_version))
assert int(tf_version[0]) >= 2, "Tensorflow version must be >= 2.0"

from sklearn.preprocessing import OneHotEncoder
import re
from sklearn.preprocessing import LabelEncoder

#seed random numbers for reproducibility
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

import itertools as it

print('\nImports Complete.')

%matplotlib inline
random.seed(0)

#### Define helper functions for processing the data

In [ ]:
cond_names = np.array(pd.read_csv('experiments_order.txt', header = None)[0]) #read in the names of expression conditions 

#parameters for finetuning
COND_IND = list(cond_names).index('Heat') #index for the first condition to try and predict with the CNN
KERNEL_SIZE = 11 #size of the kernel to predict
SEQ_LENGTH = 1000 #length of upstream sequence from which to predict, must be 20, 50 or 100
dropout_rate = 0.1
l2_lambda = 0.001
learning_rate = 1e-04
METRICS = [
      K.metrics.TruePositives(name='tp'),
      K.metrics.FalsePositives(name='fp'),
      K.metrics.TrueNegatives(name='tn'),
      K.metrics.FalseNegatives(name='fn'), 
      K.metrics.BinaryAccuracy(name='accuracy'),
      K.metrics.Precision(name='precision'),
      K.metrics.Recall(name='recall'),
      K.metrics.AUC(name='auc'),
]

# One-hot encode DNA sequence data
def one_hot_encode(seq):
    one_hot_dict = {'A': np.array([1,0,0,0]),
                   'C': np.array([0,1,0,0]),
                   'G': np.array([0,0,1,0]),
                   'T': np.array([0,0,0,1])}
    return np.array([one_hot_dict[x] for x in seq])

# Return the one-hot encoded sequence and corresponding expression data for a given upstream sequence length
# The upstream sequence length must already be present in the data file
def process_seqs_by_length (data_file, seq_length, expression_column = 'Norm Expression'):
    #read in data
    data = pd.read_csv(data_file)#['Norm Expression'][1]
    try:
        data = data.dropna(subset=[str(seq_length) + 'bp upstream sequence']) #removes genes without upstream sequences
    except:
        raise Exception ("Can't get data for sequence of length %s" % seq_length)
    expressions = np.array(data[expression_column]) # imports numerical data as strings
    
    seq_strings = data[str(seq_length) + 'bp upstream sequence'].values #INPUT, upstream sequences
    try:
        labels = np.array([[float(expr) for expr in sample[1:-1].split(', ')] for sample in expressions]) #convert expression data into arrays of floats
    except:
        labels = np.array([[float(expr) for expr in re.split('\n ?| +', sample[1:-1].strip()) if expr != ''] for sample in expressions])
    bin_labels = (labels > 0).astype('int64') # binarized labels (1 if expressed more than average, 0 if expressed less than average)
    seq_one_hot = np.array([one_hot_encode(seq) for seq in seq_strings])
    return seq_one_hot, labels, bin_labels

# Split the data seet into training and validation and return the input data and labels
def train_val_split (seq_one_hot, labels, cond_ind = COND_IND, train_frac = 0.8):
    
    arr_shape = seq_one_hot.shape
    data_split = int(arr_shape[0]*train_frac)
    
    if cond_ind:
        labels = labels[:, COND_IND] #pick which growth condition to predict growth for

    rand_ind = np.random.choice(range(arr_shape[0]), arr_shape[0], replace = False)
    train_inds = rand_ind[:data_split]
    val_inds = rand_ind[data_split:]

    t_shape = seq_one_hot[train_inds,:,:].shape
    v_shape = seq_one_hot[val_inds,:,:].shape

    x_train = tf.reshape(seq_one_hot[train_inds,:,:], (t_shape[0],1,t_shape[1],t_shape[2]))
    y_train = labels[train_inds]
    x_val = tf.reshape(seq_one_hot[val_inds,:,:], (v_shape[0],1,v_shape[1],v_shape[2]))
    y_val = labels[val_inds]
    return x_train, y_train, x_val, y_val

#### Define helper functions for building a more complex or a simpler CNN model

In [ ]:
# Returns a Keras Sequential model with 3 convolutional layers, 2 hidden fully connected layers, and
# a prediction layer
def build_comp_model(input_shape, l2_lambda, dropout_rate, output_size = 1, output_activation='sigmoid') :
    model = K.Sequential()
    model.add(K.layers.Conv2D(
        filters=30, kernel_size=(1, 20), strides=(1, 1), padding='valid', 
        kernel_regularizer='l2', activation = 'relu', input_shape = (input_shape)
    ))
    model.add(K.layers.MaxPool2D(pool_size=(1,3)))
    model.add(K.layers.Conv2D(
        filters=20, kernel_size=(1, 6), strides=(1, 1), padding='valid', 
        kernel_regularizer='l2', activation = 'relu', input_shape = (x_train.shape[1:])
    ))
    
    model.add(K.layers.MaxPool2D(pool_size=(1,3)))
    model.add(K.layers.Conv2D(
        filters=20, kernel_size=(1, 4), strides=(1, 1), padding='valid', 
        kernel_regularizer='l2', activation = 'relu', input_shape = (x_train.shape[1:])
    ))
    model.add(K.layers.MaxPool2D(pool_size=(1,3)))

    model.add(K.layers.Flatten())
    model.add(K.layers.Dense(500, activation = 'relu', kernel_regularizer=K.regularizers.l2(l2_lambda)))
    model.add(K.layers.Dropout(dropout_rate))
    model.add(K.layers.Dense(500, activation = 'relu', kernel_regularizer=K.regularizers.l2(l2_lambda)))
    model.add(K.layers.Dropout(dropout_rate))
    model.add(K.layers.Dense(output_size, activation=output_activation))
    return model

# Returns a Keras Sequential model with 1 convolutional layer, 1 hidden fully connected layers, and
# a prediction layer
def build_simp_model(input_shape, l2_lambda, dropout_rate, output_size = 1, output_activation='sigmoid', fc_layers=500) :
    model = K.Sequential()
    model.add(K.layers.Conv2D(
        filters=5, kernel_size=(1, 5), strides=(1, 1), padding='valid', 
        kernel_regularizer='l2', activation = 'relu', input_shape = (input_shape)
    ))
    model.add(K.layers.MaxPool2D(pool_size=(1,3)))
    model.add(K.layers.Flatten())
    model.add(K.layers.Dense(fc_layers, activation = 'relu', kernel_regularizer=K.regularizers.l2(l2_lambda)))
    model.add(K.layers.Dropout(dropout_rate))
    model.add(K.layers.Dense(output_size, activation=output_activation))
    return model

#### Define helper functions for grid search and model evaluation

In [ ]:
# Perform a grid search across l2 coefficients, optimizer learning rates, and upstream sequence lengths
# The loss and accuracy functions can be customized
def grid_search(build_model,
                reg_coeffs,
                learning_rates,
                seq_lengths,
                dropout_rate,
                batch_size,
                num_epochs,
                training_data,
                val_data, 
                metrics,
                loss = 'binary_crossentropy',
                loss_name = 'val_loss',
                acc_name = 'val_accuracy',
               **kwargs):

    losses = []
    accs = []
    hyperparam_combos = list(
        it.product(seq_lengths, reg_coeffs, learning_rates))
    
    
    for seq_length, reg_coeff, learning_rate in hyperparam_combos:
        x_train, y_train = training_data[seq_length]
        x_val, y_val = val_data[seq_length]
        model = build_model(x_train.shape[1:], reg_coeff, dropout_rate, **kwargs)
        
        optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate)
        
        model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        
        history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    validation_data=(x_val, y_val))
        best_loss = min(history.history[loss_name])
        best_acc = max(history.history[acc_name])
        
        losses.append([seq_length, 
                       reg_coeff, 
                       learning_rate, 
                       best_loss])
        
        accs.append([seq_length, 
                       reg_coeff, 
                       learning_rate, 
                       best_acc])
        
        
    losses_df = pd.DataFrame(losses, 
                             columns=['seq_length',
                                      'L2 lambda',
                                      'learning rate',
                                      loss_name])    
    accs_df = pd.DataFrame(accs, 
                             columns=['seq_length',
                                      'L2 lambda',
                                      'learning rate',
                                       acc_name])    
    return losses_df, accs_df

## Binary Classification 

#### Get training and validation data and labels

In [ ]:
seq_one_hot, labels, bin_labels  = process_seqs_by_length ('cleaned_data.csv', SEQ_LENGTH)
x_train, y_train, x_val, y_val = train_val_split (seq_one_hot, bin_labels)

print ('X train %s' % x_train.shape)
print ('Y train %s' % y_train.shape)
print ('X validation %s' % x_val.shape)
print ('Y validation %s'% y_val.shape)

##### Test building and training complex model

In [ ]:
learning_rate = 1e-04
model = build_comp_model(x_train.shape[1:], l2_lambda, dropout_rate)
optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=METRICS)
model.summary()

#### Test fitting model for a single expression point

In [ ]:
print('# Fit model on training data')
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    validation_data=(x_val, y_val))

#### Perform a grid search on hyperparameters for the simpler model
The complex model did not perform better than random guessing

In [ ]:
#parameters to try
reg_coeffs = [0, 1e-02, 1e-01]
learning_rates = [1e-05, 1e-04, 1e-03]
seq_lengths = [20, 50, 100, 200, 500, 1000]
batch_size = 128
num_epochs = 10

training_data = {}
val_data = {}
for length in seq_lengths:
    seq_one_hot, _, bin_labels  = process_seqs_by_length ('cleaned_data.csv', length)
    x_train, y_train, x_val, y_val = train_val_split (seq_one_hot, bin_labels)
    training_data[length] = (x_train, y_train)
    val_data[length] = (x_val, y_val)

loss_df, acc_df = grid_search(build_simp_model,
                reg_coeffs,
                learning_rates,
                seq_lengths,
                dropout_rate,
                batch_size,
                num_epochs,
                training_data,
                val_data, 
                METRICS)

In [ ]:
acc_pivot = (acc_df
                     .pivot_table(values=['val_accuracy'],
                                  columns=['L2 lambda'],
                                  index=['seq_length', 'learning rate']))
acc_pivot.style.format('{:.3f}').background_gradient(cmap='magma_r',
                                                             axis=None)

In [ ]:
loss_pivot = (loss_df
                     .pivot_table(values=['val_loss'],
                                  columns=['L2 lambda'],
                                  index=['seq_length', 'learning rate']))
loss_pivot.style.format('{:.3f}').background_gradient(cmap='magma_r',
                                                             axis=None)

#### Using hyperparameters from the grid search, train a model for each experimental growth condition to see if there are differences

In [ ]:
best_l2 = 0
best_lr = 0.001
best_seq_length = 200

best_seq_one_hot, _, best_bin_labels  = process_seqs_by_length ('cleaned_data.csv', best_seq_length)

#unpacker = {0:np.array([1,0]), 1:np.array([0,1])}
#unpacked_bin_label = np.array([np.array([unpacker[x] for x in condition]) for condition in best_bin_labels])
x_train, y_train, x_val, y_val = train_val_split(best_seq_one_hot, best_bin_labels, cond_ind = None)
optimizer = tf.keras.optimizers.SGD(learning_rate = best_lr)

best_val_acc = [] #for each condition, stores the validation loss from the best model trained for that condition
best_val_loss = []
best_val_auc = []
for i in range(len(y_train[0])):
    model_2 = build_simp_model(x_train.shape[1:], best_l2, dropout_rate, fc_layers=100)
    model_2.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=METRICS)
    
    history_2 = model_2.fit(x_train, y_train[:, i],
                        batch_size=128,
                        epochs=10,
                        validation_data=(x_val, y_val[:, i]))
    
    best_val_acc.append(np.max(history_2.history['val_accuracy']))
    best_val_loss.append(np.min(history_2.history['val_loss']))
    best_val_auc.append(np.max(history_2.history['val_auc']))

In [ ]:
sort_ls = np.flip(np.argsort(best_val_auc))
print ('Best predicted condition names and accuracy: ')
print (dict(zip((list(cond_names[sort_ls[:10]])), list(np.array(best_val_auc)[sort_ls[:10]]))))
print 
print ('Worst predicted condition names and accuracy: ')
print (dict(zip((list(cond_names[sort_ls[-9:]])), list(np.array(best_val_auc)[sort_ls[-9:]]))))

#### Visualize class imbalance across the different experimental conditions

In [ ]:
frac_overexpressed = np.sum(y_val, axis=0)/ y_val.shape[0] # per condition, fraction of genes that are overexpressed
plt.hist(frac_overexpressed)
plt.show()

#### Show that any difference in accuracy between models is due to class imbalance in training data

In [ ]:
frac_one_category = np.array(list(map(lambda x : max(x, 1-x), frac_overexpressed)))
plt.figure(figsize = (10,10))
plt.scatter(frac_one_category, best_val_acc)
plt.xlabel('Fraction of entries with the same label', fontsize = 20)
plt.ylabel('Fraction of genes correctly labeled in a condition', fontsize = 20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
#plt.savefig('figures/acc-label_scatter.png')
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
plt.hist(best_val_auc)
plt.xlabel("Estimated Area under the ROC curve", fontsize=20)
plt.ylabel("Number of Conditions", fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
#plt.savefig('figures/AUC_hist.png')
plt.show()

## Regression 

#### Test training a single simple CNN regression model

In [ ]:
learning_rate = 1e-04

reg_metrics = [
      K.metrics.MeanSquaredError(name='accuracy')
]

seq_one_hot, raw_expr, _ = process_seqs_by_length ('cleaned_data.csv', 100, expression_column = 'Expression')

x_train, y_train, x_val, y_val = train_val_split(seq_one_hot, raw_expr, cond_ind=None)

model_reg = build_simp_model(x_train.shape[1:], l2_lambda, dropout_rate, len(labels[0]), None, 1000)
optimizer = tf.keras.optimizers.SGD(learning_rate = learning_rate)
model_reg.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])

reg_history = model_reg.fit(x_train, y_train,
                    batch_size=128,
                    epochs=100,
                    validation_data=(x_val, y_val))

#### Perform a grid search for hyperparameters for the regression model

In [ ]:
#parameters to try
reg_coeffs = [0, 1e-02, 1e-01]
learning_rates = [1e-04, 1e-03]
seq_lengths = [20, 50, 100, 200, 500, 1000]
batch_size = 128
num_epochs = 100

reg_training_data = {}
reg_val_data = {}
for length in seq_lengths:
    seq_one_hot, raw_expr, _ = process_seqs_by_length ('cleaned_data.csv', length, expression_column = 'Expression')
    x_train, y_train, x_val, y_val = train_val_split (seq_one_hot, raw_expr, cond_ind=None)
    reg_training_data[length] = (x_train, y_train)
    reg_val_data[length] = (x_val, y_val)
    
loss_df, _ = grid_search(build_simp_model,
                    reg_coeffs,
                    learning_rates,
                    seq_lengths,
                    dropout_rate,
                    batch_size,
                    num_epochs,
                    reg_training_data,
                    reg_val_data, 
                    reg_metrics,
                    loss = 'mean_squared_error',
                    loss_name = 'val_accuracy',
                    acc_name = 'val_loss',
                    output_size = len(labels[0]), output_activation=None, fc_layers=500)

In [ ]:
loss_df['Mean Squared Error'] = loss_df['val_accuracy']
reg_loss_pivot = (loss_df
                     .pivot_table(values=['Mean Squared Error'],
                                  columns=['L2 lambda'],
                                  index=['seq_length', 'learning rate']))
reg_loss_pivot.style.format('{:.3f}').background_gradient(cmap='magma_r',
                                                             axis=None)

In [ ]:
best_l2 = 0.01
best_lr = 0.001
best_len = 500
x_train, y_train = reg_training_data[best_len]
x_val, y_val = reg_val_data[best_len]

model_reg = build_simp_model(x_train.shape[1:], best_l2, dropout_rate, len(labels[0]), None, 500)
optimizer = tf.keras.optimizers.SGD(learning_rate = best_lr)
model_reg.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error', K.metrics.RootMeanSquaredError()])

In [ ]:
reg_history_opt = model_reg.fit(x_train, y_train,
                    batch_size=128,
                    epochs=20,
                    validation_data=(x_val, y_val))

In [ ]:
y_val_hat = model_reg.predict(x_val)

#### Regression model accurately captures the mean of the distribution

In [ ]:
plt.figure(figsize = (10,7))
plt.plot(np.mean(y_val, axis=0), label = 'Mean experimental gene expression')
plt.plot(np.mean(y_val_hat, axis=0), label='Mean predicted gene expression')
plt.xlabel('Experimental condition index', fontsize=18)
plt.ylabel('Experimental condition index', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(fontsize=15)
#plt.savefig('figures/mean_expression.png')
plt.show()

#### Regression model fails to make meaningful predictions

In [ ]:
print ('RMSE', reg_history_opt.history['val_root_mean_squared_error'][-1] )
print ('NRMSE', reg_history_opt.history['val_root_mean_squared_error'][-1]/np.mean(y_val) )
print ('normalized mean standard deviation', np.mean(np.sqrt(np.mean(np.square(y_val - y_val_hat), axis=0))/np.mean(y_val) ))

In [ ]:
for i in range (8,16):
    plt.figure()
    plt.plot(y_val[i], label='real')
    plt.plot(y_val_hat[i], label='predicted')
    plt.legend()
    plt.show()